In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
%matplotlib inline

In [2]:
sealion_types = ["adult_males", 
    "subadult_males",
    "adult_females",
    "juveniles",
    "pups"]

In [3]:
sealion_sizes_px = {
    "adult_males": 72,
    "subadult_males": 64,
    "adult_females": 56,
    "juveniles": 32,
    "pups": 20
}

In [4]:
def gaussian2d(x, y, sx=1, sy=1, x0=0, y0=0, A=1):
    sxx = 2 * sx * sx
    syy = 2 * sy * sy
    xc = x - x0
    yc = y - y0
    xcc = xc * xc
    ycc = yc * yc
    return A * np.exp(-(xcc/sxx + ycc/syy))

In [15]:
def mask_gaussian(size):
    sigma = size / 3.0  # 97% of the gaussian in the size * size square
    half = int((sigma * 5 ) * 2)  # to be sure that the limit of the mask have a value close to 0 (avoid artifact on density map)
    im = np.zeros((2 * half + 1, 2 * half + 1), np.float32)
    for y in range(-half, half + 1):
        for x in range(-half, half + 1):
            im[y + half, x + half] = gaussian2d(x, y, sigma, sigma, 0, 0, 1)
    total_sum = np.sum(im)
    im /= total_sum
    return im

In [16]:
sealion_gaussian = {}
for sealion, size in sealion_sizes_px.items():
    sealion_gaussian[sealion] = mask_gaussian(size)

In [17]:
def create_density_map(shape, dots, mask):
    im = np.zeros(shape, np.float32)
    hmask, wmask = mask.shape
    hmask2 = hmask//2
    wmask2 = wmask//2
    
    for (xf, yf) in dots:
        x = int(round(xf))
        y = int(round(yf))
        xmin = max(0, x - wmask2)
        xmax = min(im.shape[1] - 1, x + wmask2)
        ymin = max(0, y - hmask2)
        ymax = min(im.shape[0] - 1, y + hmask2)
       
        dy_top = y - ymin
        dy_bottom = ymax - y
        dx_left = x - xmin
        dx_right = xmax - x
        im[ymin:ymax+1, xmin:xmax+1] += mask[hmask2 - dy_top: hmask2 + dy_bottom + 1, wmask2 - dx_left: wmask2 + dx_right + 1]
    return im

In [18]:
def show_density(im, density):
    max_density = np.max(density)
    normalized_density = density/max_density
    im_fp32 = im.astype(np.float32)
    im_masked = im_fp32.copy()
    im_masked[:,:,0] = (im_fp32[:,:,0] * normalized_density)
    im_masked[:,:,1] = (im_fp32[:,:,1] * normalized_density)
    im_masked[:,:,2] = (im_fp32[:,:,2] * normalized_density)
    return im_masked.astype(np.uint8)

In [19]:
with open("../data/sealion/TrainDots/872.pkl", "rb") as ifile:
    dots = pickle.load(ifile)

In [20]:
im_train = cv2.imread("../data/sealion/Train/872.jpg")

In [21]:
density_female = create_density_map(im_train.shape[:-1], dots['adult_females'], sealion_gaussian['adult_females'])

In [22]:
im_density = show_density(im_train, density_female)

In [23]:
cv2.imwrite("../data/imk.png", im_density)

True

In [24]:
np.sum(density_female)

131.99989